In [1]:
import re
from tika import parser #https://github.com/chrismattmann/tika-python
import shutil
from os import listdir
from os.path import isfile, join
import os
from datetime import date
import pandas as pd

In [2]:
base = os.getcwd()
source = os.path.join(base, 'tika\\')
output = os.path.join(base, 'gold\\')

In [3]:
# 폴더내 pdf파일을 txt 파일로 변경
def txt2data (source_folder) :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    result_list = []
    for txt in txt_files:
        a = str(txt).split('(')[2][:-5]
        date_li = a.split('.')
        time = date(int(date_li[0]), int(date_li[1]), int(date_li[2]))
        
        # 파일에서 텍스트 가져오기
        filepath = (source_folder + txt)
        with open(filepath, 'r', encoding='utf-8') as f :
            text = f.read()
            text_list = text.split('회의경과')
            text_t = text_list[1]
        new_dict = {
            'date' : time,
            'text' : text_t
        }
        result_list.append(new_dict)
    
    BASE_DIR = os.getcwd()
    SAVE_DB_DIR = os.path.join(BASE_DIR, '금통위data')

    if not os.path.exists(SAVE_DB_DIR):
        os.makedirs(SAVE_DB_DIR)

    #텍스트 파일 만들기
    temp_name = "금통위 의사록2.xlsx"
    txt_name = os.path.join(SAVE_DB_DIR, temp_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')

    df = pd.DataFrame(result_list, columns = ['date', 'text'])
    df.to_excel(excel_writer)

    excel_writer.save()

# 텍스트 파일에서 특ㅅ정 섹션만 분리하기

In [4]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text


import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)|｢?.?외환.?국제금융\s?동향.?｣?과 관련하여.*', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은|｢?.?금융시장\s?동향.?｣?과 관련하여,?', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts


In [5]:
base

'C:\\Users\\student\\Documents\\Daikoku_Python'

In [6]:
base = os.getcwd()
source = os.path.join(base, 'tika\\')
output = os.path.join(base, 'gold\\')

In [7]:
def txt_split(source_folder):
    result_list = []
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    for txt in txt_files:
        # 날자 데이터 가져오기
        a = str(txt).split('(')[2][:-5]
        date_li = a.split('.')
        time = date(int(date_li[0]), int(date_li[1]), int(date_li[2]))
        
        # 파일에서 텍스트 가져오기
        filepath = (source_folder + txt)
        with open(filepath, 'r', encoding='utf-8') as f :
            text = f.read()

        sections, section_texts = preprocess_minutes(text)
        section_2 = section_texts[1]
        section_3 = section_texts[2]
        new_dict = {
            'date' : time,
            's2' : section_2,
            's3' : section_3
        }
        result_list.append(new_dict)
    
    df = pd.DataFrame(result_list)
    return df

In [8]:
df = txt_split(source)

In [9]:
df.head()

,date,s2,s3
0,2005-06-09,[],[]
1,2005-06-23,[],[]
2,2005-07-07,[],[]
3,2005-08-11,[],[]
4,2005-09-08,[],[]


In [10]:
import numpy as np
np_df = np.array(df)

In [11]:
count = 0
for a, b, c in np_df:
    if b != []:
        count +=1
    if c != []:
        count +=1
print(count)

144


In [12]:
def make_txtlist(file):
    result_list = []
    for a, b, c in file:
        if b != []:
            new_dict = {
                'date' : a,
                'text' : ','.join(b)
            }
            result_list.append(new_dict)
        if c != []:
            new_dict = {
                'date' : a,
                'text' : ','.join(c)
            }
            result_list.append(new_dict)
    return result_list

In [13]:
text_li = make_txtlist(np_df)

In [28]:
df_txt = pd.DataFrame(text_li)
df_txt.head()

,date,text
0,2006-06-08,일부 위원은 세계적인 긴축기조의 영향으로 신흥시장(emerging market)에 ...
1,2006-09-07,일부 위원은 최근 원화선물옵션의 시카고상업거래소(CME: Chicago Mercan...
2,2008-07-10,일부 위원은 최근 통화량 증가에 대한 우려가 많이 제기되고 있다면서 M2의 경우 개...
3,2008-09-11,"일부 위원은 지난달에 기업들이 순발행, CP｢ ｣을 줄이고 조달금리가 상대적으로 높..."
4,2010-05-12,일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...


In [29]:
df_txt['token'] = 0
df_txt['ngram'] = 0
df_txt.head()

,date,text,token,ngram
0,2006-06-08,일부 위원은 세계적인 긴축기조의 영향으로 신흥시장(emerging market)에 ...,0,0
1,2006-09-07,일부 위원은 최근 원화선물옵션의 시카고상업거래소(CME: Chicago Mercan...,0,0
2,2008-07-10,일부 위원은 최근 통화량 증가에 대한 우려가 많이 제기되고 있다면서 M2의 경우 개...,0,0
3,2008-09-11,"일부 위원은 지난달에 기업들이 순발행, CP｢ ｣을 줄이고 조달금리가 상대적으로 높...",0,0
4,2010-05-12,일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,0,0


# n- gram

In [20]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [30]:
for i in range(len(df_txt)):
    txt = df_txt.text[i]

    ngrams = text2ngram(txt)
    ngram_text = ','.join(ngrams)

    df_txt.ngram[i] = ngram_text

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
df_txt.head()

,date,text,token,ngram
0,2006-06-08,일부 위원은 세계적인 긴축기조의 영향으로 신흥시장(emerging market)에 ...,0,"경상/NNG;수지/NNG;자본/NNG;수지/NNG;흑자/NNG,수출기업/NNG;환율..."
1,2006-09-07,일부 위원은 최근 원화선물옵션의 시카고상업거래소(CME: Chicago Mercan...,0,
2,2008-07-10,일부 위원은 최근 통화량 증가에 대한 우려가 많이 제기되고 있다면서 M2의 경우 개...,0,"통화/NNG;증가/NNG,투자/NNG;늘/VV"
3,2008-09-11,"일부 위원은 지난달에 기업들이 순발행, CP｢ ｣을 줄이고 조달금리가 상대적으로 높...",0,"민간/NNG;신용/NNG;증가/NNG;둔화/NNG,주택담보대출/NNG;연체율/NNG..."
4,2010-05-12,일부 위원은 남유럽사태 이후 각국이·｢ ｣금융기관에 대한 미시 거시 건전성감독의 중...,0,"외환보유액/NNG;경상/NNG;수지/NNG;흑자/NNG,둔화/NNG;물가/NNG;하..."


In [32]:
df_txt.to_excel('금통위회의록.xlsx')